In [30]:
!pip install tslearn
!pip install pandas_profiling

     |████████████████████████████████| 532kB 9.4MB/s 
  Created wheel for tslearn: filename=tslearn-0.2.5-cp36-cp36m-linux_x86_64.whl size=816768 sha256=4e17dc4af6a8caec89e6a6ec8c124be25ff07c5cb4231095b1447dd7888a1638
  Stored in directory: /root/.cache/pip/wheels/1d/6a/f7/145f874cbaebf801a5d89b71197585db9b1ddc72d666236cc3
Successfully built tslearn


In [23]:
from google.colab import drive
import pandas as pd
import numpy as np
import pandas_profiling

DRIVENAME = '/content/drive/'
FILENAME_train = "/content/drive/My Drive/problem_set_challenges_for_job_postings/EV_files/EV_train.csv"
FILENAME_train_labels = "/content/drive/My Drive/problem_set_challenges_for_job_postings/EV_files/EV_train_labels.csv"
sample_output = '/content/drive/My Drive/problem_set_challenges_for_job_postings/EV_files/sample_submission.csv'
drive.mount(DRIVENAME)

df_train = pd.read_csv(FILENAME_train, header='infer')
df_train.head()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


,House ID,Interval_1,Interval_2,Interval_3,Interval_4,Interval_5,Interval_6,Interval_7,Interval_8,Interval_9,Interval_10,Interval_11,Interval_12,Interval_13,Interval_14,Interval_15,Interval_16,Interval_17,Interval_18,Interval_19,Interval_20,Interval_21,Interval_22,Interval_23,Interval_24,Interval_25,Interval_26,Interval_27,Interval_28,Interval_29,Interval_30,Interval_31,Interval_32,Interval_33,Interval_34,Interval_35,Interval_36,Interval_37,Interval_38,Interval_39,...,Interval_2841,Interval_2842,Interval_2843,Interval_2844,Interval_2845,Interval_2846,Interval_2847,Interval_2848,Interval_2849,Interval_2850,Interval_2851,Interval_2852,Interval_2853,Interval_2854,Interval_2855,Interval_2856,Interval_2857,Interval_2858,Interval_2859,Interval_2860,Interval_2861,Interval_2862,Interval_2863,Interval_2864,Interval_2865,Interval_2866,Interval_2867,Interval_2868,Interval_2869,Interval_2870,Interval_2871,Interval_2872,Interval_2873,Interval_2874,Interval_2875,Interval_2876,Interval_2877,Interval_2878,Interval_2879,Interval_2880
0,11655099,0.950,0.826,0.361,0.238,0.342,0.233000,0.351000,0.194000,0.292000,0.234,0.260,0.274,0.192,0.329,0.192,0.443,0.260,0.222,0.419,0.249,0.429,0.426,0.373,0.335,0.457,1.199,1.735,1.431,1.151,1.258,1.243,0.825,0.534,0.513,0.775,0.708,0.655,0.618,0.600,...,0.457,0.456,0.529,0.295,0.488,0.458,0.366,0.355,0.186,0.300,0.181,0.276,0.219,0.219,0.265,0.179,0.299,0.181,0.383,0.573,0.304,0.473,0.389,0.703,0.924,0.856,0.690,0.568,0.791,0.752,0.664,0.783,0.601,0.639,0.417,0.439,0.226,0.190,0.710,0.728
1,11633257,0.353,0.327,0.358,0.292,0.285,0.304000,0.361000,0.342000,0.355000,0.348,0.342,0.261,0.276,0.334,0.411,0.269,0.338,0.818,0.914,0.835,0.852,0.911,0.919,0.917,0.905,0.898,0.844,0.892,0.873,0.865,0.860,0.388,0.298,0.413,0.350,0.345,0.281,0.411,0.442,...,0.369,0.349,0.339,0.353,0.473,0.512,0.397,0.513,0.565,0.834,1.105,1.105,0.951,0.965,1.133,0.968,1.254,0.999,0.934,0.917,1.008,1.003,1.599,0.696,0.863,0.813,0.581,0.748,0.655,0.654,0.536,0.558,0.622,0.634,0.513,0.421,0.273,0.296,0.291,0.289
2,11651552,0.150,0.181,0.150,0.150,0.131,0.125000,0.088000,0.106000,0.094000,1.019,0.094,0.394,0.375,0.444,0.150,0.713,0.175,0.175,0.131,0.700,0.288,0.200,0.244,0.819,0.500,0.231,0.231,0.225,0.200,0.163,0.200,0.344,0.388,1.550,3.694,2.738,1.463,1.175,1.675,...,0.094,0.425,0.650,0.119,0.194,0.281,0.825,1.169,0.463,0.450,1.719,0.688,0.181,0.181,0.238,0.306,0.306,0.244,0.225,0.356,0.188,1.063,0.150,0.194,0.856,0.463,0.863,1.938,1.494,1.106,2.125,0.881,0.481,1.194,0.138,0.119,0.038,0.088,0.056,0.113
3,11636092,2.088,2.075,2.121,2.098,2.046,2.081000,1.847000,0.420000,0.399000,0.364,0.413,0.363,0.397,0.377,0.971,0.966,0.930,1.155,1.285,1.207,1.166,1.173,0.693,0.467,0.595,1.866,0.747,0.550,0.744,0.693,1.145,1.140,1.113,1.080,1.338,2.540,1.300,1.911,3.071,...,0.356,0.308,0.344,0.315,0.336,0.322,0.815,0.962,1.071,0.980,0.941,0.938,0.927,0.968,0.555,0.534,0.426,0.472,0.441,0.456,0.502,0.426,0.943,1.017,1.040,2.158,2.348,2.157,1.158,0.960,0.620,0.487,0.563,0.419,0.379,0.359,0.347,0.325,0.330,0.340
4,11647239,1.416,1.250,1.270,1.258,1.239,1.753105,4.609256,4.619256,4.075151,1.230,1.192,1.212,1.195,1.184,1.193,1.182,1.176,1.196,1.202,1.248,1.209,1.218,1.216,1.214,1.228,1.193,1.188,1.203,1.190,1.200,1.183,1.185,1.181,1.189,1.197,1.183,1.225,1.223,1.313,...,1.220,1.222,1.241,1.306,1.230,1.234,1.260,1.194,1.204,1.200,1.192,1.233,1.290,1.227,1.218,1.227,1.216,1.187,1.202,1.207,1.200,1.257,1.245,1.241,1.247,1.208,1.203,1.231,1.205,1.630,1.596,1.667,1.569,1.664,1.580,1.635,1.568,1.565,1.575,1.571


In [24]:
z = pd.read_csv(sample_output)
z.head()

,House ID,Interval_1,Interval_2,Interval_3,Interval_4,Interval_5,Interval_6,Interval_7,Interval_8,Interval_9,Interval_10,Interval_11,Interval_12,Interval_13,Interval_14,Interval_15,Interval_16,Interval_17,Interval_18,Interval_19,Interval_20,Interval_21,Interval_22,Interval_23,Interval_24,Interval_25,Interval_26,Interval_27,Interval_28,Interval_29,Interval_30,Interval_31,Interval_32,Interval_33,Interval_34,Interval_35,Interval_36,Interval_37,Interval_38,Interval_39,...,Interval_2841,Interval_2842,Interval_2843,Interval_2844,Interval_2845,Interval_2846,Interval_2847,Interval_2848,Interval_2849,Interval_2850,Interval_2851,Interval_2852,Interval_2853,Interval_2854,Interval_2855,Interval_2856,Interval_2857,Interval_2858,Interval_2859,Interval_2860,Interval_2861,Interval_2862,Interval_2863,Interval_2864,Interval_2865,Interval_2866,Interval_2867,Interval_2868,Interval_2869,Interval_2870,Interval_2871,Interval_2872,Interval_2873,Interval_2874,Interval_2875,Interval_2876,Interval_2877,Interval_2878,Interval_2879,Interval_2880
0,11643173,0.602697,0.693063,0.893275,0.689664,0.841234,0.878549,0.287212,0.366362,0.230408,0.005331,0.399815,0.090541,0.284498,0.751487,0.150202,0.355765,0.696219,0.433701,0.356134,0.868834,0.158286,0.112264,0.855946,0.049759,0.386847,0.287638,0.701421,0.348359,0.101311,0.515244,0.967740,0.099007,0.563501,0.201811,0.863026,0.771011,0.623471,0.004768,0.813952,...,0.890831,0.343712,0.397915,0.052738,0.493009,0.519640,0.328790,0.988561,0.228803,0.394622,0.661096,0.229464,0.376776,0.982282,0.332578,0.336689,0.738585,0.898626,0.116646,0.837088,0.937019,0.190755,0.791627,0.155867,0.698598,0.611266,0.079596,0.828842,0.070327,0.269708,0.038768,0.493408,0.715839,0.728717,0.204594,0.786541,0.636030,0.413730,0.627268,0.961402
1,11655020,0.186095,0.242335,0.914697,0.970818,0.113643,0.732318,0.071198,0.666764,0.302415,0.428575,0.813186,0.502561,0.530760,0.101231,0.626596,0.405568,0.538856,0.369988,0.753559,0.812334,0.383748,0.019842,0.266524,0.522839,0.216698,0.878975,0.239622,0.533031,0.851856,0.777644,0.766090,0.675093,0.155563,0.820342,0.134428,0.795507,0.762913,0.499141,0.001519,...,0.772676,0.503066,0.403963,0.985349,0.345761,0.641959,0.118376,0.203554,0.501058,0.229345,0.169687,0.617088,0.143320,0.605009,0.635555,0.826432,0.694213,0.802055,0.898080,0.571430,0.418422,0.750860,0.028945,0.631760,0.702056,0.241926,0.535453,0.679453,0.925703,0.959869,0.656757,0.737361,0.503128,0.027549,0.193535,0.129298,0.206721,0.379348,0.571068,0.500008
2,11644367,0.278665,0.944197,0.612130,0.091324,0.893314,0.378808,0.502404,0.310496,0.331500,0.749801,0.142223,0.065390,0.404909,0.603602,0.262520,0.067850,0.538405,0.865503,0.317951,0.237801,0.001433,0.854960,0.810720,0.195162,0.836021,0.647052,0.382896,0.688942,0.690352,0.060783,0.427374,0.308072,0.132410,0.733835,0.412469,0.331528,0.257060,0.339601,0.172145,...,0.135086,0.253816,0.712739,0.446999,0.126497,0.174313,0.032041,0.306995,0.998362,0.111778,0.118361,0.014757,0.154904,0.749813,0.178987,0.877400,0.033183,0.036865,0.454670,0.624442,0.223641,0.739106,0.760332,0.836007,0.937144,0.964687,0.936571,0.902935,0.041428,0.420294,0.738102,0.240763,0.687204,0.351782,0.306404,0.702606,0.821001,0.301134,0.221470,0.471356
3,11633356,0.714480,0.733441,0.936425,0.893105,0.570334,0.562052,0.549747,0.871113,0.043234,0.027062,0.926260,0.567968,0.556235,0.127117,0.753180,0.421468,0.033462,0.503405,0.545384,0.522539,0.268451,0.259635,0.696353,0.019317,0.173785,0.599346,0.998564,0.064525,0.737297,0.587315,0.443951,0.121654,0.028416,0.415514,0.463804,0.087513,0.807320,0.884509,0.574972,...,0.721338,0.314076,0.287202,0.352678,0.475352,0.610997,0.941381,0.117338,0.792903,0.537098,0.382154,0.114856,0.327376,0.145555,0.196023,0.884378,0.034151,0.465338,0.878848,0.035841,0.522570,0.228806,0.240897,0.360435,0.918396,0.814797,0.582673,0.581340,0.396548,0.796753,0.684724,0.013110,0.622301,0.493600,0.844591,0.703928,0.986679,0.581308,0.929910,0.152494
4,11649905,0.377413,0.644303,0.633946,0.464369,0.854585,0.08962

In [26]:
z.shape

(699, 2881)

# A. Which houses had Electric Vehicles?

In [27]:
df_train_labels = pd.read_csv(FILENAME_train_labels, header='infer')
df_train_labels['sums'] = df_train_labels.iloc[:, 1:].sum(axis=1)
has_EVs = df_train_labels['sums'] > 0
list_of_house_IDs_with_EVS = df_train_labels.loc[has_EVs, "House ID"].values
mask = df_train_labels.loc[has_EVs, "House ID"].index
list_of_house_IDs_with_EVS[:10]

array([11647239, 11655054, 11635803, 11642383, 11655167, 11643133,
       11646352, 11633451, 11640674, 11649350])

# B. When were the electric vehicles charging?

There are a few ways we can answer this:

As suggested from the post, you can answer this by providing an array of the probabilities at each time interval for when an EV would be charging.


Each row is an observation in the df_train dataset, and each observation's labels is the corresponding row in the df_train_labels dataset

In [0]:
from tslearn.utils import to_time_series, to_time_series_dataset
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from tslearn.svm import TimeSeriesSVC
from tslearn.neighbors import KNeighborsTimeSeriesClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline

In [29]:
df_train.head()

,House ID,Interval_1,Interval_2,Interval_3,Interval_4,Interval_5,Interval_6,Interval_7,Interval_8,Interval_9,Interval_10,Interval_11,Interval_12,Interval_13,Interval_14,Interval_15,Interval_16,Interval_17,Interval_18,Interval_19,Interval_20,Interval_21,Interval_22,Interval_23,Interval_24,Interval_25,Interval_26,Interval_27,Interval_28,Interval_29,Interval_30,Interval_31,Interval_32,Interval_33,Interval_34,Interval_35,Interval_36,Interval_37,Interval_38,Interval_39,...,Interval_2841,Interval_2842,Interval_2843,Interval_2844,Interval_2845,Interval_2846,Interval_2847,Interval_2848,Interval_2849,Interval_2850,Interval_2851,Interval_2852,Interval_2853,Interval_2854,Interval_2855,Interval_2856,Interval_2857,Interval_2858,Interval_2859,Interval_2860,Interval_2861,Interval_2862,Interval_2863,Interval_2864,Interval_2865,Interval_2866,Interval_2867,Interval_2868,Interval_2869,Interval_2870,Interval_2871,Interval_2872,Interval_2873,Interval_2874,Interval_2875,Interval_2876,Interval_2877,Interval_2878,Interval_2879,Interval_2880
0,11655099,0.950,0.826,0.361,0.238,0.342,0.233000,0.351000,0.194000,0.292000,0.234,0.260,0.274,0.192,0.329,0.192,0.443,0.260,0.222,0.419,0.249,0.429,0.426,0.373,0.335,0.457,1.199,1.735,1.431,1.151,1.258,1.243,0.825,0.534,0.513,0.775,0.708,0.655,0.618,0.600,...,0.457,0.456,0.529,0.295,0.488,0.458,0.366,0.355,0.186,0.300,0.181,0.276,0.219,0.219,0.265,0.179,0.299,0.181,0.383,0.573,0.304,0.473,0.389,0.703,0.924,0.856,0.690,0.568,0.791,0.752,0.664,0.783,0.601,0.639,0.417,0.439,0.226,0.190,0.710,0.728
1,11633257,0.353,0.327,0.358,0.292,0.285,0.304000,0.361000,0.342000,0.355000,0.348,0.342,0.261,0.276,0.334,0.411,0.269,0.338,0.818,0.914,0.835,0.852,0.911,0.919,0.917,0.905,0.898,0.844,0.892,0.873,0.865,0.860,0.388,0.298,0.413,0.350,0.345,0.281,0.411,0.442,...,0.369,0.349,0.339,0.353,0.473,0.512,0.397,0.513,0.565,0.834,1.105,1.105,0.951,0.965,1.133,0.968,1.254,0.999,0.934,0.917,1.008,1.003,1.599,0.696,0.863,0.813,0.581,0.748,0.655,0.654,0.536,0.558,0.622,0.634,0.513,0.421,0.273,0.296,0.291,0.289
2,11651552,0.150,0.181,0.150,0.150,0.131,0.125000,0.088000,0.106000,0.094000,1.019,0.094,0.394,0.375,0.444,0.150,0.713,0.175,0.175,0.131,0.700,0.288,0.200,0.244,0.819,0.500,0.231,0.231,0.225,0.200,0.163,0.200,0.344,0.388,1.550,3.694,2.738,1.463,1.175,1.675,...,0.094,0.425,0.650,0.119,0.194,0.281,0.825,1.169,0.463,0.450,1.719,0.688,0.181,0.181,0.238,0.306,0.306,0.244,0.225,0.356,0.188,1.063,0.150,0.194,0.856,0.463,0.863,1.938,1.494,1.106,2.125,0.881,0.481,1.194,0.138,0.119,0.038,0.088,0.056,0.113
3,11636092,2.088,2.075,2.121,2.098,2.046,2.081000,1.847000,0.420000,0.399000,0.364,0.413,0.363,0.397,0.377,0.971,0.966,0.930,1.155,1.285,1.207,1.166,1.173,0.693,0.467,0.595,1.866,0.747,0.550,0.744,0.693,1.145,1.140,1.113,1.080,1.338,2.540,1.300,1.911,3.071,...,0.356,0.308,0.344,0.315,0.336,0.322,0.815,0.962,1.071,0.980,0.941,0.938,0.927,0.968,0.555,0.534,0.426,0.472,0.441,0.456,0.502,0.426,0.943,1.017,1.040,2.158,2.348,2.157,1.158,0.960,0.620,0.487,0.563,0.419,0.379,0.359,0.347,0.325,0.330,0.340
4,11647239,1.416,1.250,1.270,1.258,1.239,1.753105,4.609256,4.619256,4.075151,1.230,1.192,1.212,1.195,1.184,1.193,1.182,1.176,1.196,1.202,1.248,1.209,1.218,1.216,1.214,1.228,1.193,1.188,1.203,1.190,1.200,1.183,1.185,1.181,1.189,1.197,1.183,1.225,1.223,1.313,...,1.220,1.222,1.241,1.306,1.230,1.234,1.260,1.194,1.204,1.200,1.192,1.233,1.290,1.227,1.218,1.227,1.216,1.187,1.202,1.207,1.200,1.257,1.245,1.241,1.247,1.208,1.203,1.231,1.205,1.630,1.596,1.667,1.569,1.664,1.580,1.635,1.568,1.565,1.575,1.571


In [0]:
from statsmodels.tsa.seasonal import seasonal_decompose

def plot_seasonal_decompose(data, model):
    '''
    Summary:
    -------
    Plots trend, seasonality and residuals with bigger plot size=(10,2)
    
    Parameters:
    ----------
    data(pd.Dataframe, pd.Series): time series data
    
    Return:
    ------
    decomposition(obj): return value of statsmodels.tsa.seasonal.seasonal_decompose
    '''
    decomposition = seasonal_decompose(data,model=model)
    decomposition.trend.plot(figsize=(10,2), title='Trend');
    decomposition.seasonal.plot(figsize=(10,2), title='Seasonality');
    decomposition.resid.plot(figsize=(10,2), title='Residuals');
    return decomposition

In [0]:
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error

for idx, row in df_train.iterrows():
    X = df_train.iloc[idx, 1:].values
    train, test = X[0:size], X[size:len(X)]
    predictions = list()
    for t in range(len(test)):
        model = ARIMA(history, order=(5,1,0))
        model_fit = model.fit(disp=0)
        output = model_fit.forecast()
        yhat = output[0]
        predictions.append(yhat)
        obs = test[t]
        history.append(obs)
        # print('predicted=%f, expected=%f' % (yhat, obs))
        
    print(len(train))
    print(len(history))
    print(len(predictions))
    predicted_row = pd.DataFrame([train, predictions])
    predictions_df.append(predicted_row)

error = mean_squared_error(test, predictions)
print('Test MSE: %.3f' % error)
# plot
plt.plot(test, color='blue')
plt.plot(predictions, color='red')
plt.legend(['Actual Values', "Predictions"])
plt.show()


1900
2880
980
1900
3860
980
1900
4840
980
1900
5820
980
1900
6800
980
1900
7780
980
1900
8760
980
1900
9740
980


/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


1900
10720
980
1900
11700
980
1900
12680
980
1900
13660
980
1900
14640
980
1900
15620
980
1900
16600
980
1900
17580
980
1900
18560
980
1900
19540
980
1900
20520
980
1900
21500
980
1900
22480
980
1900
23460
980
1900
24440
980
1900
25420
980
1900
26400
980
1900
27380
980


In [0]:
# figure settings
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)

X = df_train.iloc[4, 1:].values.ravel()
y = df_train_labels.iloc[4, 1:].values.ravel()

splits = TimeSeriesSplit(n_splits=3)
plt.figure(1)
print(splits)
model = TimeSeriesSVC()

index = 1
for train_index, test_index in splits.split(X):
    train = X[train_index]
    test = X[test_index]
    print(f"Observations: {len(train) + len(train)}")
    print(f"Train Observations: {len(train)}")
    print(f"Test Observations: {len(train)}")
    plt.subplot(310+index)
    plt.plot(train)
    plt.plot([None for i in train] + [x for x in train])
    index += 1
plt.show()

# C. Other Interesting Aspects of the dataset

Plot the time series to see if there is anything interesting to talk about

In [0]:
fig, ax = plt.subplots(figsize=(32, 12))
train_size = round(len(X)*.33)
X = df_train.iloc[0, 1:].values
X_train = X[:train_size]
X_test = X[train_size:]
print("Observations: {}".format(len(X)))
print("Training Observations: {}".format(X_train.shape))
print("Test Observations: {}".format(X_test.shape))
ax.plot(X_train)
ax.plot([None for i in X_train] + [x for x in X_test])
plt.axvline(x=len(X_train), ymin=0, ymax=1)
plt.show()

### Compare aggregate statistics between Have-EVs vs Have-Not-EVs

We hypothesize the the aggregate statistics and the statistical moments (mean, variance) of the houses with electric vehicles are higher than that of the have-not-EV household.

How you cause an exhibition of this:

a) statistics through tabulation

b) statistical tests

c) plot the pdf or the distribution of the different points and color code them

### C-a

In [0]:
#### Does not have EV #########

have_not_EVs_condition = df_train_labels['sums'] == 0
mask = df_train_labels.loc[have_not_EVs_condition, "House ID"].index
have_not_EVs_df = df_train.iloc[mask, 1:].copy()
have_not_EVs_df.iloc[0, 1:].aggregate(['mean', 'median', 'max', 'min', 'var', 'std', 'skew', 'kurt'], axis=0)

have_not_EVs_df_aggs = pd.DataFrame(have_not_EVs_df.iloc[:, 1:].agg(['mean', 'median', 'max', 'min', 'var', 'std', 'skew', 'kurt'], axis=1)) # row wise aggregations with axis=1
have_not_EVs_df_aggs.head()

In [0]:
#### Does HAVE EV #########

EV_condition = df_train_labels['sums'] > 0
mask = df_train_labels.loc[EV_condition, "House ID"].index
EV_df = df_train.iloc[mask, 1:].copy()
EV_df.iloc[0, 1:].aggregate(['mean', 'median', 'max', 'min', 'var', 'std', 'skew', 'kurt'], axis=0)

has_EV_df_aggs = pd.DataFrame(EV_df.iloc[:, 1:].agg(['mean', 'median', 'max', 'min', 'var', 'std', 'skew', 'kurt'], axis=1)) # row wise aggregations with axis=1
has_EV_df_aggs.head()

### C-c-i: Distribution Plots --- Have-Not EVs [mean, median, max]

In [0]:
fig, ax = plt.subplots(figsize=(32, 12))
plt.title('Aggregrate statistcs distribution plots for each House that does not have an EV')
plt.ylabel('Power Readings (Kh)')
plt.xlabel('Half Hour Intervals of Readings')
plt.xlim(have_not_EVs_df_aggs['mean'].min(), have_not_EVs_df_aggs['mean'].max())
sns.distplot(have_not_EVs_df_aggs['mean'], kde_kws={"color": "blue", "lw": 1, "label": "mean"})
sns.distplot(have_not_EVs_df_aggs['median'], kde_kws={"color": "red", "lw": 1, "label": "median"} )
sns.distplot(have_not_EVs_df_aggs['max'], kde_kws={"color": "green", "lw": 1, "label": "max"})

# median markers
plt.axvline(x=have_not_EVs_df_aggs['mean'].median(), ymin=0, ymax=0.35, dashes=(5, 2, 1, 2))
plt.axvline(x=have_not_EVs_df_aggs['max'].median(), ymin=0, ymax=0.35, dashes=(5, 2, 1, 2))

### C-c-ii: Distribution Plots --- Have-Not EVs [variance, skew, kurtosis]

In [0]:
fig, ax = plt.subplots(figsize=(32, 12))
plt.title('Aggregrate statistcs distribution plots for each House that has an EV')
plt.ylabel('Power Readings (Kh)')
plt.xlabel('Half Hour Intervals of Readings')

plt.xlim(have_not_EVs_df_aggs['skew'].min(), 
         have_not_EVs_df_aggs['skew'].max())

sns.distplot(have_not_EVs_df_aggs['var'], kde_kws={"color": "blue", "lw": 1, "label": "var"})
sns.distplot(have_not_EVs_df_aggs['skew'], kde_kws={"color": "red", "lw": 1, "label": "skew"} )
sns.distplot(have_not_EVs_df_aggs['kurt'], kde_kws={"color": "green", "lw": 1, "label": "kurt"})

The boxenplot below illustrates the distribution of mean usage across the two groups.

In [0]:
df_no_ev = pd.DataFrame(columns=['mean_power_usage_(Kh)', 'class_'])
df_ev = pd.DataFrame(columns=['mean_power_usage_(Kh)', 'class_'])

df_no_ev['mean_power_usage_(Kh)'] = have_not_EVs_df_aggs['mean'].values
df_no_ev['class_'] = np.zeros(have_not_EVs_df_aggs['mean'].values.shape[0], )
df_ev['mean_power_usage_(Kh)'] = pd.Series(has_EV_df_aggs['mean'].values)
df_ev['class_'] = pd.Series(np.ones(has_EV_df_aggs['mean'].values.shape[0], ))

df = pd.concat([df_no_ev, df_ev])
df.replace({0:'No EV', 1:'Has EV'}, inplace=True)

fig, ax = plt.subplots(figsize=(28, 9))
sns.boxenplot(x='class_', y='mean_power_usage_(Kh)', data=df)

### C-a: Tabulation of Aggregate Statistics --- Has EVs

In [0]:
#### Has EV ########

has_EVs_condition = df_train_labels['sums'] > 0
mask = df_train_labels.loc[has_EVs_condition, "House ID"].index
has_EV_df = df_train.iloc[mask, 1:].copy()
has_EV_df_aggs = pd.DataFrame(has_EV_df.iloc[:, 1:].agg(['mean', 'median', 'max', 'min', 'var', 'std', 'skew', 'kurt'], axis=1)) # row wise aggregations with axis=1
has_EV_df_aggs.head()

### C-b-i: Statistical Tests --- Has-EVs vs. Have-Not-EVs

Statistical tests on all aggregrate statistics between the two groups with an alpha=0.5

Null Hypothesis: There is no significant difference between the two groups for each of the aggregate statistics, meaning that the aggregate statistics between the two groups are the same

Alternative Hypothesis: There is a significant difference between the two groups for each of the aggregate statistics and that they are distinctly from different groups

In [0]:
from scipy.stats import mannwhitneyu

mannwhitneyu_mean = mannwhitneyu(has_EV_df_aggs['mean'].values, have_not_EVs_df_aggs['mean'].values)
mannwhitneyu_mean

####### CONCLUSION #########

The Mann Whitney U test indicates that we should reject the null hypothesis and conclude that the means of the samples between the two groups (has EVs and has-not EVs) are statistically significantly different given that the p-value << 0.05, U=211674).

Visually, we also see that the mean power usage across observations for houses that have an EV are greater than that of houses without EVs. This is also true for their outliers and extremes, respectively.

In [0]:
fig, ax = plt.subplots(figsize=(24, 9))
plt.title('Aggregrate statistcs distribution plots for each House that has an EV')
plt.ylabel('Power Readings (Kh)')
plt.xlim(has_EV_df_aggs['mean'].min(), has_EV_df_aggs['mean'].max())
sns.distplot(has_EV_df_aggs['mean'], kde_kws={"color": "blue", "lw": 1, "label": "Has EV mean"})
sns.distplot(have_not_EVs_df_aggs['mean'], kde_kws={"color": "red", "lw": 1, "label": "No EV mean"})

### C-b-ii: Some outlier analysis

The list of values generated below show the extreme outliers from a house that could possibly have more than 1 EV. Compared with this list of sorted means from houses that do not have EVs.



In [0]:
has_EV_df_aggs['mean'].sort_values(ascending=False)

In [0]:
have_not_EVs_df_aggs['mean'].sort_values(ascending=False)

### C-c-i: Distribution Plots --- Has EVs [mean, median, max]

Leptokurtic mean and median distribution plots.

Distribution of Max values are higher than that of the plots without EVs

In [0]:
fig, ax = plt.subplots(figsize=(32, 12))
plt.title('Aggregrate statistcs distribution plots for each House that has an EV')
plt.ylabel('Power Readings (Kh)')
plt.xlim(has_EV_df_aggs['mean'].min(), has_EV_df_aggs['mean'].max())

sns.distplot(has_EV_df_aggs['mean'], kde_kws={"color": "blue", "lw": 1, "label": "mean"})
sns.distplot(has_EV_df_aggs['median'], kde_kws={"color": "red", "lw": 1, "label": "median"} )
sns.distplot(has_EV_df_aggs['max'], kde_kws={"color": "green", "lw": 1, "label": "max"})


# median markers
plt.axvline(x=has_EV_df_aggs['mean'].median(), ymin=0, ymax=0.35, dashes=(5, 2, 1, 2))
plt.axvline(x=has_EV_df_aggs['max'].median(), ymin=0, ymax=0.35, dashes=(5, 2, 1, 2))

### C-c-ii: Distribution Plots --- Has EVs [variance, skew, kurtosis]

In [0]:
fig, ax = plt.subplots(figsize=(32, 12))
plt.title('Aggregrate statistcs distribution plots for each House that has an EV')
plt.ylabel('Power Readings (Kh)')

plt.xlim(has_EV_df_aggs['kurt'].min(), 
         has_EV_df_aggs['skew'].max())

sns.distplot(has_EV_df_aggs['var'], kde_kws={"color": "blue", "lw": 1, "label": "var"})
sns.distplot(has_EV_df_aggs['skew'], kde_kws={"color": "red", "lw": 1, "label": "skew"} )
sns.distplot(has_EV_df_aggs['kurt'], kde_kws={"color": "green", "lw": 1, "label": "kurt"})

### C-c-iii: Outlier Analysis

Here we also wee that the distribution of outlier mean values for houses with EVs are higher than that of non-EV households. Are they statistically significantly different from one another?

In [0]:
def get_outliers_and_extremes(df, numeric_attribute):
    from scipy.stats import iqr
    from numpy import percentile

    IQR = iqr(df[numeric_attribute], axis=0, rng=(25, 75), scale='raw', nan_policy='propagate', interpolation='linear', keepdims=False)
    q1 = percentile(df[numeric_attribute], 0.25, axis=0, out=None, overwrite_input=False, interpolation='linear', keepdims=False)
    q3 = percentile(df[numeric_attribute], 0.75, axis=0, out=None, overwrite_input=False, interpolation='linear', keepdims=False)
    
    outliers = df[(df[numeric_attribute] <= (q1 - (IQR * 1.5))) | (df[numeric_attribute] <= (q3 + (IQR * 1.5)))]
    extremes = df[(df[numeric_attribute] <= (q1 - (IQR * 1.5))) | (df[numeric_attribute] <= (q3 + (IQR * 1.5)))]
    return outliers

outliers_has_ev = get_outliers_and_extremes(has_EV_df_aggs, 'mean')
outliers_has_ev

In [0]:
outliers_has_NO_ev = get_outliers_and_extremes(have_not_EVs_df_aggs, 'mean')
outliers_has_NO_ev

In [0]:
fig, ax = plt.subplots(figsize=(22, 10))
sns.distplot(outliers_has_NO_ev['mean'])
sns.distplot(outliers_has_ev['mean'])
plt.legend(labels=['No EV', 'EV'])
plt.xlabel('Mean Outlier of Power Usage of Houses (Kh)')

In [0]:
def perform_mannwhitneyu_statistical_test(df_1, df_2, numeric_attribute):
    statistic, pvalue = mannwhitneyu(df_1[numeric_attribute].values, df_2[numeric_attribute].values)
    print("Results of Mann-Whitney U Test\nMWUT Statistic: {}\nP-Value: {}".format(statistic, pvalue))
    if pvalue <=0.05:
        print("We reject the null hypothesis and conclude that the mean usage values from the two groups that they are significantly different.")
    else:
        print("We fail to reject the null hypothesis, and conclude that the means between the two groups are not signficantly different from one another.")

perform_mannwhitneyu_statistical_test(outliers_has_ev, outliers_has_NO_ev, 'mean')

### C-c-iii: Analysis of Data Without Outliers

In [0]:
has_EV_no_outliers = has_EV_df_aggs.drop(index=outliers_has_ev.index, inplace=False, axis=0)
has_not_EV_no_outliers = have_not_EVs_df_aggs.drop(index=outliers_has_NO_ev.index, inplace=False, axis=0)

fig, ax = plt.subplots(figsize=(32, 12))
sns.distplot(has_EV_no_outliers['mean'])
sns.distplot(has_not_EV_no_outliers['mean'])
plt.legend(labels=['EV', 'No EV'])

In [0]:
perform_mannwhitneyu_statistical_test(has_EV_no_outliers, has_not_EV_no_outliers, 'mean')

### Intuitive
when the outliers are rem
oved (that is, the outliers in both groups including the activity periods when the EVs were charging at a house) from both groups, each groups mean power usage is not significantly different from the other's.

### Other Exploration

In [0]:
from statsmodels.tsa.seasonal import seasonal_decompose
from datetime import timedelta, datetime

index = []
start_date = datetime(2018, 10, 13, 12, 0, 0)
for td in (start_date + timedelta(minutes=30*it) for it in range(df_train.iloc[0, 1:].shape[0])):
    index.append(td.strftime("%Y-%m-%d %H:%M"))

training_data = pd.Series(df_train.iloc[0, 1:].values.flatten())
index = pd.date_range(start='1/1/2018', end='1/2/2018', periods=training_data.shape[0], freq=None, tz=None, normalize=False, name=None, closed=None)
df = pd.DataFrame(index=index)
df['power_usage_(Kh)'] = training_data.values
training_data.index = pd.DatetimeIndex(freq='w', start=0, periods=training_data.shape[0])
sea = seasonal_decompose(training_data, model='multiplicative')
sea.plot()

In [0]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller

def plot_data_properties(data, ts_plot_name="Time Series plot"):
    '''
    Summary:
    -------
    Plots various plots, including time series, autocorrelation,
    partial autocorrelation and distribution plots of data.
    
    Parameters:
    ----------
    ts_plot_name(String): The name of the time series plot
    data(pd.Dataframe, pd.Series, array): Time Series Data
    
    Returns:
    --------
    None
    
    '''
    plt.figure(figsize=(16,4))
    plt.plot(data)
    plt.title(ts_plot_name)
    plt.ylabel('Power Usage (Kh)')
    plt.xlabel('Half-Hour Intervals')
    fig, axes = plt.subplots(1,3,squeeze=False)
    fig.set_size_inches(16,4)
    plot_acf(data, ax=axes[0,0], lags=48);
    plot_pacf(data, ax=axes[0,1], lags=48);
    sns.distplot(data, ax=axes[0,2])
    axes[0,2].set_title("Probability Distribution")

training_data = pd.Series(EV_df.iloc[100, 1:].values.flatten())
plot_data_properties(training_data)

In [0]:
class StationarityTests:
    def __init__(self, significance=.05):
        self.SignificanceLevel = significance
        self.pValue = None
        self.isStationary = None

    def ADF_Stationarity_Test(self, timeseries, printResults = True):

        #Dickey-Fuller test:
        adfTest = adfuller(timeseries, autolag='AIC')
        
        self.pValue = adfTest[1]
        
        if (self.pValue<self.SignificanceLevel):
            self.isStationary = True
        else:
            self.isStationary = False
        
        if printResults:
            dfResults = pd.Series(adfTest[0:4], index=['ADF Test Statistic','P-Value','# Lags Used','# Observations Used'])

            #Add Critical Values
            for key,value in adfTest[4].items():
                dfResults['Critical Value (%s)'%key] = value

            print('Augmented Dickey-Fuller Test Results:')
            print(dfResults)

sTest = StationarityTests()
sTest.ADF_Stationarity_Test(training_data, printResults = True)
print("Is the time series stationary? {0}".format(sTest.isStationary))

In [0]:
def plot_seasonal_decompose(data, model):
    '''
    Summary:
    -------
    Plots trend, seasonality and residuals with bigger plot size=(10,2)
    
    Parameters:
    ----------
    data(pd.Dataframe, pd.Series): time series data
    
    Return:
    ------
    decomposition(obj): return value of statsmodels.tsa.seasonal.seasonal_decompose
    '''
    decomposition = seasonal_decompose(data,model=model)
    decomposition.trend.plot(figsize=(10,2), title='Trend');
    decomposition.seasonal.plot(figsize=(10,2), title='Seasonality');
    decomposition.resid.plot(figsize=(10,2), title='Residuals');
    return decomposition

## Ignore the axis labeling as I am still working on trying to figure out how to set the ticks to half hour intervals

In [0]:

training_data = pd.Series(EV_df.iloc[100, 1:].values.flatten())
training_data.index = pd.DatetimeIndex(freq="h", start=0, periods=training_data.shape[0])
dec = plot_seasonal_decompose(training_data, 'multiplicative')
dec.plot()

Recreation of the trend plot, which takes a rolling average with a window size of a week.

In [0]:
# 336 half hour intervals in a week
import matplotlib.dates as md

xformatter = md.DateFormatter('%Y-%m-%d %H:%M')
ma_336 = training_data.rolling(window=336).mean()
plt.figure(figsize=(12,4))
plt.plot(ma_336)
xlocator = md.MinuteLocator(byminute=[0,30], interval = 1)
plt.xticks(rotation=270)
plt.gcf().axes[0].xaxis.set_major_formatter(xformatter)
plt.title('MA_336')

Correlation plot between two sets of lags. Between contiguous lags, they do seem to be very correlated. The further away the temporal horizon the further more the the correlation drops.

In [0]:
fig, axes = plt.subplots(1,2, squeeze=False)
fig.set_size_inches(14,4)
axes[0,0].scatter(x=df[1:], y=df.shift(1)[1:])
axes[0,1].scatter(x=df[48:], y=df.shift(48)[48:])
axes[0,0].set_title('Correlation of y_t and y_t-1')
axes[0,1].set_title('Correlation of y_t and y_t-48')

Plot time series, ACF, PACF and PDF of a sample outlier time series

In [0]:
dataset = EV_df.iloc[outliers_has_ev.index[:10], :]
training_data = pd.Series(dataset.iloc[9, 1:].values.flatten())
plot_data_properties(training_data)

### Data preparation

If we are going to use an autoregressive model like the SARIMA model (that is linear model like linear regression), we must first perform a log-transformation of the time series (because (log a*b) = log(a)+log(b)) and thusly log(Y) = log(T)+log(S)+log(R) so that the data become linear, fittable and smoother.

In [0]:
log_transformed_data = np.log(training_data)
plot_data_properties(log_transformed_data, ts_plot_name="Log tranformed training data")

Sometimes log-transforming the data on itself makes the data stationary and here, the log-transformed time series is stationary, however, it was stationary before the log-transform.

In [0]:
sTest.ADF_Stationarity_Test(log_transformed_data, printResults = True)
print("Is the time series stationary? {0}".format(sTest.isStationary))

We perform first order differencing is another basic operation to make time series stationary. It removes time series components from the data and you are left with changes between time periods. We take away only Trend,not Seasonality. Data is stationary now. If you look at the histogram at the below graph, it looks like Normal bell curve. Stationary data is randomly i.i.d distributed and the plot looks like white noise. White noise is just an example of stationary time series data.

In [0]:
logged_diffed_data = log_transformed_data.diff()[1:]
plot_data_properties(logged_diffed_data, ts_plot_name='Log transformed and differenced data')

In [0]:
sTest.ADF_Stationarity_Test(logged_diffed_data, printResults = True)
print("Is the time series stationary? {0}".format(sTest.isStationary))

We have to take 24th order difference to remove seasonality. You may ask how we decided to take 24th order difference not 6th or 8th or other order. Usually, monthly data has seasonality at lag=12, weekly data has at lag=4 and daily has at lag=30. Or you can derive it from the ACF plot. The12th, 24th, 36th lags are highly correlated for this particular data.

In [0]:
seasonally_diffed_data = logged_diffed_data.diff(24)[24:]
plot_data_properties(seasonally_diffed_data, 'Log transformed, diff=24 and seasonally differenced data')

The Ljung-Box test is a test to see if the residuals are white noise. is a way to test for the absence of serial autocorrelation, up to a specified lag k.

Null Hypothesis: our model does not show lack of fit (or in simple terms—the model is just fine)

Alternative Hypothesis: the model does show a lack of fit.

A significant p-value in this test rejects the null hypothesis that the time series isn’t autocorrelated.

From the p-values we can say that there is a lack of fit and that the time series isn't autocorrelated.

In [0]:
from statsmodels.stats.diagnostic import acorr_ljungbox
# In an ARMA model, this value is usually p+q where p is the AR order and q is the MA order. This value is subtracted from the degrees-of-freedom
# used in the test so that the adjusted dof for the statistics are lags - model_df. If lags - model_df <= 0, then NaN is returned.
ljung_box_statistic, pvalue = acorr_ljungbox(seasonally_diffed_data, 
                                             lags=None, 
                                             boxpierce=False) # model_df = p+q which is 1+1 or if modified, m-p-q, where m=time lag
print("Ljung-Box test statistis:\n{}\nP-Values:\n{}".format(ljung_box_statistic, pvalue))

After log differencing the time series by taking the 24th order difference, we can see that without a doubt (p-value = 0.000000), that our time series is stationary (if you needed more proof, take a look at the pdf plot on the bottom row on the right).

In [0]:
sTest.ADF_Stationarity_Test(seasonally_diffed_data, printResults = True)
print("Is the time series stationary? {0}".format(sTest.isStationary))

In [0]:
p = [1]
d = 0 # 0 if taking a stationary series, else taking the order that the time series has to be differenced in order to get to stationarity; 1 if you want the model to do the differencing itself
q = [1]
s = [12, 48] # seasonality frequency, every 48 half hour intervals to make a day
Q = [1]
P = [1]
D = 0 # 1 if you want the model to do the differencing itself

In [0]:
def best_sarima_model(train_data,p,q,P,Q,d=1,D=1,s=12):
    from statsmodels.tsa.statespace.sarimax import SARIMAX
    best_model_aic = np.Inf 
    best_model_bic = np.Inf 
    best_model_hqic = np.Inf
    best_model_order = (0,0,0)
    models = []
    for p_ in p:
        for q_ in q:
            for P_ in P:
                for Q_ in Q:
                    try:
                        no_of_lower_metrics = 0
                        model = SARIMAX(endog=train_data,order=(p_,d,q_), seasonal_order=(P_,D,Q_,s),
                                        enforce_invertibility=False).fit(maxiter=200, method='powell')
                        models.append(model)
                        if model.aic <= best_model_aic: no_of_lower_metrics+=1
                        if model.bic <= best_model_bic: no_of_lower_metrics+=1
                        if model.hqic <= best_model_hqic:no_of_lower_metrics+=1
                        if no_of_lower_metrics >= 2:
                            best_model_aic = np.round(model.aic,0)
                            best_model_bic = np.round(model.bic,0)
                            best_model_hqic = np.round(model.hqic,0)
                            best_model_order = (p_,d,q_,P_,D,Q_,s)
                            current_best_model = model
                            models.append(model)
                            print("Best model so far: SARIMA" +  str(best_model_order) + 
                                  " AIC:{} BIC:{} HQIC:{}".format(best_model_aic,best_model_bic,best_model_hqic)+
                                  " resid:{}".format(np.round(np.exp(current_best_model.resid).mean(),3)))

                    except:
                        pass

    print('\n')
    print(current_best_model.summary())                
    return current_best_model, models 

In [0]:
current_best_model, models = best_sarima_model(training_data,p,q,P=P,Q=Q,d=d,D=D,s=24) # d=1 and D=1 so that the model can do the differencing itself or 0 if not

ARIMA or SARIMA models are estimated by using MLE and OLS assumptions are applicable to this family of models. I don't want to elaborate on these assumptions here. It is a topic of another article. However, we have to confirm that our model aligns with those assumptions. P values of coefficients are <= 0.05. Residuals are stationary and homoscedastic. There is no serial correlation among residuals.

MAPE (mean absolute percentage error) is a metric that is not offered by sklearn so we have to do it ourselves, but it is a good error metric to evaluate our model because it is intuitive.

![MAPE Formula](https://miro.medium.com/max/175/1*IX9aOt_e6vTXl5naOhWVTg.jpeg)

In [0]:
def mean_abs_pct_error(actual_values, forecast_values):
    err=0
    for i in range(len(forecast_values)):
        err += np.abs(actual_values.values[i] - forecast_values.values[i])/actual_values.values[i]
    return err[0] * 100/len(forecast_values)

In [0]:
# preds_best = np.exp(current_best_model.predict(start=0,end=test_data.shape[0], dynamic=True, typ='levels'))
# print("MAPE{}%".format(np.round(mean_abs_pct_error(test_data, preds_best),2)))

In [0]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

agile_model = SARIMAX(endog=training_data,order=(1,1,2), seasonal_order=(1,1,2,12),enforce_invertibility=False).fit()
agile_model.summary()

In [0]:
# agile_model_pred = np.exp(agile_model.predict(start=test_start_date,end=’2019–01–01', dynamic=True, typ=’levels’))
# print(“MAPE{}%”.format(np.round(mean_abs_pct_error(test_data,agile_model_pred),2)))